# Task 4.8

### Contents:

1. Dealing with memory issues
2. Aggregate mean order number grouped by department id for entire dataframe
3. Create a loyalty flag for existing customers
4. Analyse product price statistics for each loyalty category
5. Create a spending flag for each user based on the average price across their orders
6. Create an order frequency flag that marks the regularity of a user's ordering behaviour
7. Exporting Data

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path = r'C:\Users\lydia\OneDrive\Documents\Career Foundry Data\Data Immersion\Acheivement 4\Instacart Basket Analysis May-2023'
ords_prods_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_updated_2.pkl'))

### Dealing with memory issues

In [3]:
ords_prods_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32399732 entries, 0 to 32404858
Data columns (total 18 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   order_id               int32   
 1   user_id                int32   
 2   order_number           int8    
 3   orders_day_of_week     int8    
 4   order_time_of_day      int8    
 5   days_since_last_order  float16 
 6   product_id             int32   
 7   add_to_cart_order      int32   
 8   reordered              int8    
 9   product_name           object  
 10  aisle_id               int8    
 11  department_id          int8    
 12  prices                 float16 
 13  _merge                 category
 14  price_range_loc        object  
 15  busiest day            object  
 16  Busiest days           object  
 17  Busiest period of day  object  
dtypes: category(1), float16(2), int32(4), int8(6), object(5)
memory usage: 2.3+ GB


In [4]:
# Drop _merge 
ords_prods_merged.drop(columns = {'_merge'}, inplace = True)

In [5]:
# Change data types
ords_prods_merged['price_range_loc'] = ords_prods_merged['price_range_loc'].astype('category')
ords_prods_merged['busiest day'] = ords_prods_merged['busiest day'].astype('category')
ords_prods_merged['Busiest days'] = ords_prods_merged['Busiest days'].astype('category')
ords_prods_merged['Busiest period of day'] = ords_prods_merged['Busiest period of day'].astype('category')

In [6]:
ords_prods_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32399732 entries, 0 to 32404858
Data columns (total 17 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   order_id               int32   
 1   user_id                int32   
 2   order_number           int8    
 3   orders_day_of_week     int8    
 4   order_time_of_day      int8    
 5   days_since_last_order  float16 
 6   product_id             int32   
 7   add_to_cart_order      int32   
 8   reordered              int8    
 9   product_name           object  
 10  aisle_id               int8    
 11  department_id          int8    
 12  prices                 float16 
 13  price_range_loc        category
 14  busiest day            category
 15  Busiest days           category
 16  Busiest period of day  category
dtypes: category(4), float16(2), int32(4), int8(6), object(1)
memory usage: 1.4+ GB


###  Aggregate mean order number grouped by department id for entire dataframe

In [7]:
# Group the data by department id and then aggregate the mean order number for each department
ords_prods_merged.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


In [8]:
# Import department dataframe to assess what each department corresponds to
df_depart = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'departments_wrangled.csv'))

In [9]:
df_depart

,Unnamed: 0,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


Observation:

The results show that department 21 has the highest mean order number (however, this department corresponds to missing values). The actual departments with the highest mean order numbers are 10, 18 and 4 which correspond to bulk, babies and produce. 

These results differ from the subset in that all departments are displayed here. In addition, there is less variation in the difference between order number means. For example, in the subset the lowest mean was 11, which was a lot less than the others. Whereas in the entire dataframe all the means are within 15 and 22. 


### Create a loyalty flag for existing customers

In [10]:
# Creating a new column displaying max orders for certain user_id
ords_prods_merged['max_order'] = ords_prods_merged.groupby(['user_id'])['order_number'].transform(np.max)

In [11]:
ords_prods_merged.head(10)

,order_id,user_id,order_number,orders_day_of_week,order_time_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest day,Busiest days,Busiest period of day,max_order
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid range product,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid range product,Regularly busy,Least busy days,Average orders,10
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid range product,Regularly busy,Least busy days,Most orders,10
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid range product,Least busy,Least busy days,Average orders,10
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid range product,Least busy,Least busy days,Most orders,10
5,3367565,1,6,2,7,19.0,196,1,1,Soda,77,7,9.0,Mid range product,Regularly busy,Regularly busy,Average orders,10
6,550135,1,7,1,9,20.0,196,1,1,Soda,77,7,9.0,Mid range product,Regularly busy,Busiest days,Most orders,10
7,3108588,1,8,1,14,14.0,196,2,1,Soda,77,7,9.0,Mid range product,Regularly busy,Busiest days,Most orders,10
8,2295261,1,9,1,16,0.0,196,4,1,Soda,77,7,9.0,Mid range product,Regularly busy,Busiest days,Most orders,10
9,2550362,1,10,4,8,30.0,196,1,1,Soda,77,7,9.0,Mid range product,Least busy,Least busy days,Average orders,10


In [12]:
# Changing display to not limit the number of rows 
pd.options.display.max_rows = None

In [13]:
ords_prods_merged.head(50)

,order_id,user_id,order_number,orders_day_of_week,order_time_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest day,Busiest days,Busiest period of day,max_order
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid range product,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid range product,Regularly busy,Least busy days,Average orders,10
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid range product,Regularly busy,Least busy days,Most orders,10
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid range product,Least busy,Least busy days,Average orders,10
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid range product,Least busy,Least busy days,Most orders,10
5,3367565,1,6,2,7,19.0,196,1,1,Soda,77,7,9.0,Mid range product,Regularly busy,Regularly busy,Average orders,10
6,550135,1,7,1,9,20.0,196,1,1,Soda,77,7,9.0,Mid range product,Regularly busy,Busiest days,Most orders,10
7,3108588,1,8,1,14,14.0,196,2,1,Soda,77,7,9.0,Mid range product,Regularly busy,Busiest days,Most orders,10
8,2295261,1,9,1,16,0.0,196,4,1,Soda,77,7,9.0,Mid range product,Regularly busy,Busiest days,Most orders,10
9,2550362,1,10,4,8,30.0,196,1,1,Soda,77,7,9.0,Mid range product,Least busy,Least busy days,Average orders,10


In [14]:
# Creating loyalty comlumn based on max order criteria
ords_prods_merged.loc[ords_prods_merged['max_order'] > 40, 'loyalty_flag'] = 'Loyal Customer'
ords_prods_merged.loc[(ords_prods_merged['max_order'] <= 40) & (ords_prods_merged['max_order'] > 10), 'loyalty_flag'] = 'Regular Customer'
ords_prods_merged.loc[ords_prods_merged['max_order'] <= 10, 'loyalty_flag'] = 'New Customer'

In [15]:
ords_prods_merged['loyalty_flag'].value_counts()

Regular Customer    15874128
Loyal Customer      10282763
New Customer         6242841
Name: loyalty_flag, dtype: int64

### Analyse product price statistics for each loyalty category

In [16]:
ords_prods_merged.groupby('prices')

In [17]:
# Group data based on the 'loyalty_flag' column and agg basic price stats
ords_prods_merged.groupby('loyalty_flag').agg({'prices':['mean']})

,prices
,mean
loyalty_flag,
Loyal Customer,7.773427
New Customer,7.801095
Regular Customer,7.798138


In [18]:
ords_prods_merged['prices'].max()

25.0

### Create a spending flag for each user based on the average price across their orders

In [19]:
# Split the data into groups based on the 'user_id' and create a column for 'mean_prices'
ords_prods_merged['mean_prices'] = ords_prods_merged.groupby(['user_id'])['prices'].transform(np.mean)

In [20]:
# Creating spending_flag column based on mean_prices criteria
ords_prods_merged.loc[ords_prods_merged['mean_prices'] < 10, 'spending_flag'] = 'Low Spender'
ords_prods_merged.loc[ords_prods_merged['mean_prices'] >= 10, 'spending_flag'] = 'High Spender'

In [21]:
# Assess value counts
ords_prods_merged['spending_flag'].value_counts(dropna = False)

Low Spender     32280272
High Spender      119460
Name: spending_flag, dtype: int64

### Create an order frequency flag that marks the regularity of a user's ordering behaviour

In [22]:
# Split the data into groups based on the 'user_id' and create a column for 'median days since last order'
ords_prods_merged['median_days_prior_order'] = ords_prods_merged.groupby(['user_id'])['days_since_last_order'].transform(np.median)

In [23]:
# Check
ords_prods_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_time_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_range_loc,busiest day,Busiest days,Busiest period of day,max_order,loyalty_flag,mean_prices,spending_flag,median_days_prior_order
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,9.0,Mid range product,Regularly busy,Regularly busy,Average orders,10,New Customer,6.367535,Low Spender,20.5
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,9.0,Mid range product,Regularly busy,Least busy days,Average orders,10,New Customer,6.367535,Low Spender,20.5
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,9.0,Mid range product,Regularly busy,Least busy days,Most orders,10,New Customer,6.367535,Low Spender,20.5
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,9.0,Mid range product,Least busy,Least busy days,Average orders,10,New Customer,6.367535,Low Spender,20.5
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,9.0,Mid range product,Least busy,Least busy days,Most orders,10,New Customer,6.367535,Low Spender,20.5


In [24]:
# Creating spending_flag column based on mean_prices criteria
ords_prods_merged.loc[ords_prods_merged['median_days_prior_order'] > 20, 'order_freq_flag'] = 'Non-frequent customer'
ords_prods_merged.loc[(ords_prods_merged['median_days_prior_order'] > 10) & (ords_prods_merged['median_days_prior_order'] <= 20), 'order_freq_flag'] = 'Regular customer'
ords_prods_merged.loc[ords_prods_merged['median_days_prior_order'] <= 10, 'order_freq_flag'] = 'Frequent customer'

In [25]:
ords_prods_merged['order_freq_flag'].value_counts()

Frequent customer        21556644
Regular customer          7207340
Non-frequent customer     3635743
Name: order_freq_flag, dtype: int64

### Exporting Data

In [26]:
ords_prods_merged.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'order_product_no_outliers.pkl'))